Transaction Table *

* TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
* TransactionAMT: transaction payment amount in USD
* ProductCD: product code, the product for each transaction
* card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
* addr: address
* dist: distance
* P_ and (R__) emaildomain: purchaser and recipient email domain
* C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
* D1-D15: timedelta, such as days between previous transaction, etc.
* M1-M9: match, such as names on card and address, etc.
* Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

Categorical Features:

* ProductCD
* card1 - card6
* addr1, addr2
* P_emaildomain
* R_emaildomain
* M1 - M9

Identity Table *

Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions.
They're collected by Vesta’s fraud protection system and digital security partners.
(The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)

* Categorical Features:
* DeviceType
* DeviceInfo
* id_12 - id_38



In [1]:
import pandas as pd
import numpy as np
import gc
import re
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import OneClassSVM
from matplotlib import rcParams
# figure size in inches
rcParams['figure.figsize'] = 12, 10

In [ ]:
train_identity = pd.read_csv('data/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv('data/ieee-fraud-detection/train_transaction.csv')

In [ ]:
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
train = train.drop(columns=[x for x in train.columns if x.startswith("V")])
train.shape

In [ ]:
del train_identity, train_transaction
gc.collect()

In [ ]:
test_identity = pd.read_csv('data/ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv('data/ieee-fraud-detection/test_transaction.csv')

In [ ]:
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')
test = test.drop(columns=[x for x in test.columns if x.startswith("V")])
test.shape

In [ ]:
del test_identity, test_transaction
gc.collect()

In [ ]:
test["isFraud"] = "test"

In [ ]:
train.shape, test.shape

In [ ]:
for col in list(train.columns):
    if col not in list(test.columns):
        print(" only in train: ", col, end = "  ")
for col in list(test.columns):
    if col not in list(train.columns):
        print(" only in test: ", col, end = "  ") 

In [ ]:
test.columns=test.columns.str.replace('-','_')

In [ ]:
for col in list(train.columns):
    if col not in list(test.columns):
        print(" only in train: ", col, end = "  ")
for col in list(test.columns):
    if col not in list(train.columns):
        print(" only in test: ", col, end = "  ") 

In [ ]:
train.shape, test.shape

In [ ]:
list(set(train.columns) - set(test.columns))

In [ ]:
# bigdata = train.append(test, ignore_index=True)
# bigdata.shape

In [ ]:
# dict(bigdata.isna().any())

In [ ]:
# print(bigdata.sample(5).T.to_string())


In [ ]:
# dict(zip(bigdata.columns, bigdata.isna().sum()))

In [ ]:
frauds = train.shape[0]
yes = train['isFraud'].sum()
no = frauds - yes
print('{} frauds'.format(frauds))
print('{} yes \n{} no'.format(yes,no))

# remove columns over 80% missing

In [ ]:
missing_perc = (train.isna().sum() / train.isna().count()).sort_values(ascending=False)
dict(missing_perc)

In [ ]:
perc_too_high_cols = missing_perc[missing_perc > 0.80].index
perc_too_high_cols

In [ ]:
print(train.shape, test.shape)
train.drop(columns=perc_too_high_cols, inplace=True)
test.drop(columns=perc_too_high_cols, inplace=True)
print(train.shape, test.shape)

In [8]:
# save train and test
# train.to_pickle("train1.pkl")
# test.to_pickle("test1.pkl")
train = pd.read_pickle("train1.pkl")
list(train.columns)

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'P_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D10',
 'D11',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9']

## Remove too many NaN on rows

In [6]:
def drop_more_than_N_Nan(N): # of the 68 columns, only 12 allowed to be NaN
    print(train.shape)
    train.dropna(thresh=train.shape[1]-N, inplace=True)
    print(train.shape)
    
drop_more_than_N_Nan(12)

(590540, 46)
(512130, 46)


# Prepare Train 

In [7]:
train.set_index("TransactionID", inplace=True)
train.shape, train.columns

((512130, 45),
 Index(['isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1',
        'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
        'P_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
        'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D10',
        'D11', 'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'],
       dtype='object'))

## check categories

In [ ]:
for col, values in train.loc[:, train.dtypes == object].iteritems():
    num_uniques = values.nunique()
    if num_uniques > 5:
        print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))
        print (values.unique()[:20], "....")
        print ("")
        print (values.value_counts()[:10])
        print ('\n')

In [ ]:
train[["P_emaildomain", "R_emaildomain"]]

## add new column from emails which are the same

In [ ]:
train['is_email_same'] = train.apply(lambda x: x['P_emaildomain']== x['R_emaildomain'] and x['R_emaildomain'] != np.nan, axis=1)

In [ ]:
train['is_email_same'].value_counts()

In [ ]:
train[train['P_emaildomain'] == train['R_emaildomain']].filter(["P_emaildomain","R_emaildomain"])

In [ ]:
train['P_emaildomain'].value_counts()


In [ ]:
train.groupby(["P_emaildomain", "isFraud"]).size().unstack(level=1)

In [ ]:
train.groupby(["P_emaildomain"]).apply(lambda x: x['isFraud'].sum()/x['isFraud'].count())

In [ ]:
def get_perc(x):
    return x.sum()/x.count()

In [ ]:
train.groupby(["P_emaildomain"]).agg(P_emaildomain_PercFraud=("isFraud", get_perc)).reset_index().sort_values("P_emaildomain_PercFraud", ascending=False)

In [ ]:
email_repl_dict = {re.compile('.*gmail.*'): 'gmail',
             re.compile('.*hotmail.*'): 'hotmail',
             re.compile('.*yahoo.*'): 'yahoo',
             re.compile('.*aol.*'): 'aol',
             re.compile('.*anonymous.*'): 'anonymous',
             re.compile('.*outlook.*'): 'outlook',
             re.compile('.*protonmail.*'): 'protonmail'}

In [ ]:
# train['P_emaildomain'].replace(email_repl_dict, regex=True, inplace=True)
# train['R_emaildomain'].replace(email_repl_dict, regex=True, inplace=True)
d = defaultdict(lambda: 'other')
d.update(dict(zip(email_repl_dict.values(), email_repl_dict.values())))
train['P_emaildomain'] = train['P_emaildomain'].replace(email_repl_dict, regex=True, inplace=False).map(d)
train['P_emaildomain'].value_counts().head(15)


In [ ]:
train['R_emaildomain'] = train['R_emaildomain'].replace(email_repl_dict, regex=True, inplace=False).map(d)
train['R_emaildomain'].value_counts().head(15)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(train.groupby(["id_31"]).agg({"isFraud":[get_perc, "count"]}, as_index=False).reset_index()[['id_31','isFraud']].sort_values(('isFraud','count'), ascending=False))

In [ ]:
# id_30_repl_dict = {re.compile('.*ndroid.*'): 'android',
#              re.compile('.*indows.*'): 'windows',
#              re.compile('.*iOS.*'): 'ios',
#              re.compile('.*OS X.*'): 'mac',
#              re.compile('.*Mac*'): 'mac'
#                   }

In [ ]:
# d = defaultdict(lambda: 'other')
# d.update(dict(zip(id_30_repl_dict.values(), id_30_repl_dict.values())))
# train['id_30'].replace(id_30_repl_dict, regex=True, inplace=False).map(d).value_counts().head(15)

In [ ]:
# train.groupby(["DeviceInfo","isFraud"]).size().to_frame('size').reset_index().sort_values("size", ascending=False)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(train.groupby(["DeviceInfo"]).agg({"isFraud":[get_perc, "count"]}, as_index=False).reset_index()[['DeviceInfo','isFraud']].sort_values(('isFraud','count'), ascending=False))

In [ ]:
DeviceInfo_repl_dict = {re.compile('.*rv.*'): 'rv',
             re.compile('.*indows.*'): 'windows',
             re.compile('.*iOS.*'): 'ios',
             re.compile('.*Build.*'): 'build',
             re.compile('.*rident.*'): 'trident',
             re.compile('.*Moto.*'): 'moto'
                  }

In [ ]:
d = defaultdict(lambda: 'other')
d.update(dict(zip(DeviceInfo_repl_dict.values(), DeviceInfo_repl_dict.values())))
train['DeviceInfo'] = train['DeviceInfo'].replace(DeviceInfo_repl_dict, regex=True, inplace=False).map(d)
train['DeviceInfo'].value_counts().head(15)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(train.groupby(["id_31"]).agg({"isFraud":[get_perc, "count"]}, as_index=False).reset_index()[['id_31','isFraud']].sort_values(('isFraud','get_perc'), ascending=False))

In [ ]:
id_31_repl_dict = {re.compile('.*icedragon.*'): 'icedragon',
             re.compile('.*irefox.*'): 'firefox',
             re.compile('.*android.*'): 'android',
             re.compile('.*chrome.*'): 'chrome',
             re.compile('.*safari.*'): 'safari',
             re.compile('.*ie.*'): 'ie',
             re.compile('.*edge.*'): 'ie',
             re.compile('.*samsung browser.*'): 'samsung',
             re.compile('.*opera*'): 'opera'
                  }

In [ ]:
d = defaultdict(lambda: 'other')
d.update(dict(zip(id_31_repl_dict.values(), id_31_repl_dict.values())))
train['id_31'] = train['id_31'].replace(id_31_repl_dict, regex=True, inplace=False).map(d)
train['id_31'].value_counts().head(15)

In [ ]:
f = plt.figure(figsize=(19, 15))
plt.matshow(train.corr(), fignum=f.number)
plt.xticks(range(train.select_dtypes(['number']).shape[1]), train.select_dtypes(['number']).columns, fontsize=14, rotation=45)
plt.yticks(range(train.select_dtypes(['number']).shape[1]), train.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16);

In [ ]:
corrMatrix = train.corr()
sns.heatmap(corrMatrix, annot=False, cmap="Greens")
plt.show()

as we can see no features are highly correlated if the isFruad, because that would make it too easy :) 

However card3 and is_email_same show promise

In [ ]:
print(train.sample(5).T.to_string())

In [ ]:
# (train
#     .filter(corrMatrix.columns)
#      .query("isFraud == 1")
#      .drop('isFraud', axis=1)
# )

I don't think we have ordinal values, but I need to encode to use them 

In [ ]:
x = []
for col in train.columns:
    vals = train[col].value_counts().shape[0]
    missing = round((train[col].isna().sum()/train[col].shape[0])*100,2)
    x.append([col,vals,missing])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pd.DataFrame(x, columns=["col","vals","missing"]).set_index("col"))

In [ ]:
# list(train.columns)
list(train.loc[:,train.dtypes != object])

In [ ]:
train.to_pickle("train2.pkl")

# scale everything

In [ ]:
num_cols_mask = (train.dtypes != object) & (train.nunique()>5)
num_cols = train.columns[num_cols_mask]
train.loc[:, num_cols_mask]

In [ ]:
scaler = StandardScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])

In [ ]:
train[num_cols]

In [ ]:
train[num_cols].describe().round(3).T.sort_values("max", ascending=False)

In [ ]:
train[num_cols].diff().hist(color="k", alpha=0.5, bins=50, figsize=(25,25));


Remove NaN

# fill missing and remove outliers

In [ ]:
print(train.shape)
imp_kNN = KNNImputer(missing_values = np.nan)
imp_kNN.fit(train[num_cols])
train[num_cols] = imp_kNN.transform(train[num_cols])
print(train.shape)

In [ ]:
# print dataframe.
fig = plt.figure()
ax = fig.add_subplot(121)
ax.hist(np.log(train["TransactionAmt"]), bins=100);
ax = fig.add_subplot(122)
ax.hist(train["TransactionAmt"], bins=100);

# Hold Out Validation

In [ ]:
train